In [ ]:

import os
import subprocess
import sys
import getpass
from pathlib import Path

# --- CONFIGURAÇÃO ---
raw_source = "./"
# Se o path começar com "/media~/", substitui "~" pelo seu usuário em /media
username = getpass.getuser()
if raw_source.startswith("/media/~/"):
    corrected = raw_source.replace("/media/~/", f"/media/{username}/")
else:
    corrected = raw_source
# Agora expande "~" se ainda existir
SOURCE_DIR = Path(os.path.expanduser(corrected))
OUTPUT_DIR = SOURCE_DIR / "pdfs"
PYTHON_EXE = sys.executable
# ---------------------

# Verifica pasta de origem
if not SOURCE_DIR.exists():
    raise FileNotFoundError(f"Pasta de origem não encontrada: {SOURCE_DIR}")

# Cria apenas a pasta pdfs dentro de SOURCE_DIR, sem recriar o resto
OUTPUT_DIR.mkdir(exist_ok=True)

# Converte cada .ipynb em PDF
for nb_path in SOURCE_DIR.glob("*.ipynb"):
    pdf_name = nb_path.stem + ".pdf"
    cmd = [
        PYTHON_EXE, "-m", "jupyter", "nbconvert",
        str(nb_path),
        "--to", "pdf",
        "--output", pdf_name,
        "--output-dir", str(OUTPUT_DIR),
        "--debug"
    ]
    print(f"🔄 Convertendo {nb_path.name} → {pdf_name}")
    try:
        out = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(out.stdout)
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro ao converter {nb_path.name}:\n{e.stderr}")
print("✅ Conversão de todos os notebooks finalizada.")


In [ ]:
# Analisando os documentos escreva uma apresentação detalhada do estudo, em alto nível.
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata




# Analisando os documentos descreva a metodologia e os resultados que foram obtidos no estudo.
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata




# Analisando os documentos descreva a analise e discussão e os resultados que foram obtidos no estudo.
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata




# Analisando os documentos descreva as conclusões do estudo
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata


# Agora com base nos texto gerado nas etapas anteriores quero um relatório completo.
# Este relatório deve comunicar os achados para os tomadores de decisão
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata
# Estrutura do relatório: Apresentação (4 paragrafos), metodologia e resultados(8 paragrafos), análise e discussão(4 paragrafos), conclusão(2 paragrafos)





In [ ]:
# Analisando os documentos extraia os codigos que geram visualizações.
# Adapte estes codigos para salvar as imagens em PNG na pasta "figuras"
# Nomear estas Figuras em ordem, figura_1.png, figura_2.png, figura_3.png, et
# Retornar um codigo unico que gere e salve todas as imangens em png na pasta correta

In [3]:
from src.libs.lib import *
import pandas as pd
import matplotlib.pyplot as plt
df_original = pd.read_csv("src/data/days_until_first_contract_and_price")
#df_original = df_original[df_original["type"] == "Coleta/Entrega"]
df_original = df_original[df_original["type"] == "Diária"]
df_original = df_original[df_original["week"] < 8]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_original = df_original.sort_values("week")
display(df_original.shape[0])

Nome do estudo: 7 - AHA! Moment - Otimização da taxa de retenção
Hash numérica de 5 dígitos: 05259
Execução em: 2025-04-17 17:10:16


2276

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Garantir que a pasta "figuras" exista
os.makedirs("figuras", exist_ok=True)

# Cópia do DataFrame original e preparação dos dados
df = df_original.copy()
features = ['total_contracts_until_week', 'total_value_until_week', 'retention']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[features])
pca = PCA(n_components=1)
principal_components = pca.fit_transform(scaled_features)
df['PC1'] = principal_components
df['PC1_quartile'] = pd.qcut(df['PC1'], q=4, labels=["Q1", "Q2", "Q3", "Q4"])
df_stats = df.copy()

# Gráficos de barras com barra de erro
grouped = df.groupby('PC1_quartile').agg({
    'week': ['mean', 'std'],
    'total_contracts_until_week': ['mean', 'std'],
    'total_value_until_week': ['mean', 'std'],
    'retention': ['mean', 'std']
}).reset_index()

variaveis = ['week', 'total_contracts_until_week', 'total_value_until_week', 'retention']

fig_num = 1
for var in variaveis:
    plt.figure(figsize=(8, 6))
    plt.bar(grouped['PC1_quartile'], grouped[(var, 'mean')],
            yerr=grouped[(var, 'std')], capsize=5, color='skyblue')
    plt.title(f'{var} por Quartil com Barra de Erro')
    plt.xlabel('Quartil')
    plt.ylabel(var)
    plt.ylim(bottom=0)
    plt.savefig(f'figuras/figura_{fig_num}.png')
    plt.close()
    fig_num += 1

# Gráfico de retenção com baseline
df_stats_grouped = df_stats.groupby("PC1_quartile")[["week", 
    "total_contracts_until_week", "total_value_until_week", "retention"]].agg(['mean', 'std']).reset_index()
baseline = df_stats_grouped[('retention', 'mean')].mean()

plt.figure(figsize=(8, 6))
plt.bar(df_stats_grouped['PC1_quartile'], df_stats_grouped[('retention', 'mean')],
        yerr=df_stats_grouped[('retention', 'std')], capsize=5, color='skyblue', label='Retention')
plt.axhline(baseline, color='red', linestyle='--', label=f'Baseline ({baseline:.2f})')
plt.title('Gráfico de Barras - Retention por Quartil')
plt.xlabel('Quartil')
plt.ylabel('Retention')
plt.ylim(bottom=0)
plt.legend()
plt.savefig(f'figuras/figura_{fig_num}.png')
plt.close()
fig_num += 1

# Curvas Gaussianas para cada variável por quartil
stats = df.groupby("PC1_quartile")[["week", "total_contracts_until_week", 
    "total_value_until_week", "retention"]].agg(['mean', 'std', 'count']).reset_index()

for var in variaveis:
    plt.figure(figsize=(8, 6))
    for idx, row in stats.iterrows():
        mu = row[(var, 'mean')]
        sigma = row[(var, 'std')]
        n = row[(var, 'count')]
        if sigma == 0:
            sigma = 1e-6
        x = np.linspace(mu - 3*sigma, mu + 3*sigma, 200)
        y = (n / (sigma * np.sqrt(2*np.pi))) * np.exp(-((x - mu)**2) / (2*sigma**2))
        quartile_value = str(row['PC1_quartile'])
        label = f"{quartile_value} - média: {mu:.2f}"
        line, = plt.plot(x, y, label=label)
        plt.fill_between(x, y, alpha=0.3, color=line.get_color())
        plt.axvline(mu, color=line.get_color(), linestyle='--')

    plt.title(f"Curvas Gaussianas para '{var}' por PC1_quartile")
    plt.xlabel(var)
    plt.ylabel("Densidade Ajustada")
    plt.ylim(bottom=0)
    plt.legend()
    plt.savefig(f'figuras/figura_{fig_num}.png')
    plt.close()
    fig_num += 1


/tmp/ipykernel_213167/655440169.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('PC1_quartile').agg({
/tmp/ipykernel_213167/655440169.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_stats_grouped = df_stats.groupby("PC1_quartile")[["week",
/tmp/ipykernel_213167/655440169.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  stats = df.groupby("PC1_quartile")[["week", "total_contracts_unti

In [ ]:
# Analisando o relatório gerado na etapa anterior quero que gere um código python para gerar um fluxograma
# Incluir neste fluxograma as etapas do estudo
# Incluir os principais resultados
# Incluir os principais achados
# Incluir as principais conclusões
# O fluxograma deve ser salvo na pasta "figuras" como um arquivo png
# O nome do arquivo deve ser "fluxograma.png"
# O código deve ser único e gerar o fluxograma completo
# O código deve ser executável e gerar o fluxograma completo
# Garantir que a imagem não fique muito pesada
# Código de referencia: from graphviz import Digraph# Fluxograma acadêmico estilizadoflow = Digraph('Academic_Flowchart', format='png')# Atributos do grafo (layout)flow.attr('graph',           rankdir='TB',           # Direção top-to-bottom para estilo acadêmico          fontsize='12',           fontname='Times New Roman',          bgcolor='white',           margin='0.2')# Atributos dos nós (estilo acadêmico)flow.attr('node',           shape='rectangle',           style='rounded,filled',           fillcolor='#F7F9FB',    # Tom suave          color='#2E4053',        # Borda escura          fontname='Times New Roman',           fontsize='12',           margin='0.2')# Atributos das arestas (linhas)flow.attr('edge',           color='#2E4053',           arrowhead='vee',           penwidth='1.5')# Definição das etapasetapas = [    ('A', 'Início'),    ('B', '1. Importação de Dados\n(pandas)'),    ('C', '2. Pré-processamento\n(numpy, unidecode)'),    ('D', '3. Estatística Descritiva\n(statsmodels)'),    ('E', '4. Modelagem Preditiva\n(scikit-learn)'),    ('F', '5. Visualização de Dados\n(matplotlib, seaborn)'),    ('G', '6. Exportação de Relatórios\n(openpyxl, tabulate)'),    ('H', '7. Deploy & BQ\n(functions-framework,\ngoogle-cloud-bigquery)'),    ('I', 'Conclusão')]# Cria os nósfor key, label in etapas:    flow.node(key, label)# Conecta as etapas em sequênciafor (src, _), (dst, _) in zip(etapas, etapas[1:]):    flow.edge(src, dst)# Exibe o fluxogramaflow

In [7]:
from graphviz import Digraph
import os

# Garante que a pasta "figuras" exista
os.makedirs("figuras", exist_ok=True)

# Cria o fluxograma com layout acadêmico
flow = Digraph('Fluxograma_Estudo', format='png')
flow.attr(dpi='300')  # Alta resolução para qualidade de imagem

# Layout do grafo
flow.attr('graph',
          rankdir='TB',
          fontsize='12',
          fontname='Times New Roman',
          bgcolor='white',
          margin='0.2')

# Estilo base dos nós
flow.attr('node',
          shape='rectangle',
          style='rounded,filled',
          fillcolor='#F7F9FB',
          color='#34495E',
          fontname='Times New Roman',
          fontsize='12',
          margin='0.15')

# Estilo das conexões
flow.attr('edge',
          color='#2E4053',
          arrowhead='vee',
          penwidth='1.5')

# Cores para destacar seções específicas
cores_etapas = {
    'análise': '#E8F6F3',
    'visualização': '#FEF9E7',
    'resultado': '#FADBD8',
    'conclusao': '#E8DAEF'
}

# Fonte destacada para resultados e conclusões
font_destaque = {'fontsize': '14', 'fontname': 'Times New Roman Bold'}

# Etapas do estudo com customizações
etapas = [
    ('A', '🧠 Início do Estudo\nObjetivo: Melhorar Retenção'),
    ('B', 'Importação e Filtragem de Dados\nApenas motoristas Diária, semana < 8'),
    ('C', 'Criação da Métrica de Retenção\n(retention = 1 se ativo, 0 se churn)'),
    ('D', 'Aplicação de PCA\nRedução dimensional para identificação de padrões'),
    ('E', 'Classificação em Quartis\nCom base na componente principal PC1'),
    ('F', 'Análise Estatística\nComparação entre quartis'),
    ('G', '📈 Visualização Gráfica\nGráficos de barras e curvas gaussianas'),
    ('H', '📊 Principais Resultados:\n• Q1: 9,9% de retenção\n• Q4: 99,6% de retenção'),
    ('I', '💡 Achados:\n• Sucesso inicial prevê retenção\n• Receita e nº contratos são decisivos'),
    ('J', '📝 Conclusões:\n• Intervenções nas primeiras semanas são estratégicas\n• Retenção pode ser otimizada com suporte inicial'),
    ('K', '🏁 Fim do Estudo')
]

# Adiciona os nós com estilos específicos
for key, label in etapas:
    if key == 'F':
        flow.node(key, label, fillcolor=cores_etapas['análise'])
    elif key == 'G':
        flow.node(key, label, fillcolor=cores_etapas['visualização'])
    elif key == 'H':
        flow.node(key, label, fillcolor=cores_etapas['resultado'], **font_destaque)
    elif key == 'J':
        flow.node(key, label, fillcolor=cores_etapas['conclusao'], **font_destaque)
    else:
        flow.node(key, label)

# Conecta os nós sequencialmente
for (src, _), (dst, _) in zip(etapas, etapas[1:]):
    flow.edge(src, dst)

# Renderiza o arquivo na pasta "figuras"
flow.render('figuras/fluxograma', cleanup=True)


'figuras/fluxograma.png'

In [ ]:

# Agora com base no relatório completo quero que gere um documento HTML para ele. 
# Não usar Python nesta etapa.
# Gerar em HTMl puramente.
# Usar as normas da ABNT
# Neste documento incluir o texto adaptado do relatório. 
# Incluir também as figuras geradas na etapa anterior. com titulos e descrição.
# Incluir e citar no texto as figuras a partir dos PNGs gerados na etapa anterior
# Também incluir o fluxograma gerado na etapa anterior.
# Deixar responsivo
